XGBoost作为一种提升树，通过集成多棵树，对数据具有极好的泛化能力。XGBoost可以很好地处理缺失值，
对于特征的值有缺失的样本，XGBoost采用的稀疏感知算法可以自动学习出它的分裂方向。现将该算法的学习路径总结如下：

* 构造目标函数——泰勒级数近似
* 模型参数化
* 树的复杂度
* 寻找最佳分裂点——贪心算法、近似算法
* 稀疏感知算法

具体的算法推导过程可以参考

链接：https://mp.weixin.qq.com/s/HDEKnIufbW8xQcOgHaXlZw

In [57]:
#加载需要的模块
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error,  make_scorer
from xgboost.sklearn import XGBRegressor
import xgboost as xgb
from lightgbm.sklearn import LGBMRegressor

In [2]:
#导入数据
data = pd.read_csv('F:/data/used_car_train_20200313.csv', sep=' ') #sep=' '表示原数据是以空格分隔

In [62]:
data.shape

(150000, 5)

In [3]:
data=data[['v_12','v_10','v_9','v_11','price']]
#选择前面特征工程过程中筛选出的特征

In [51]:
#划分标签
x = data.drop("price",axis=1)
y = data["price"]

In [52]:
#交叉验证
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.3) #划分训练集train和验证集val

In [55]:
#设置模型参数
def build_model_xgb(x_train,y_train):
    model = xgb.XGBRegressor(n_estimators=150, learning_rate=0.1, gamma=0, subsample=0.8,\
        colsample_bytree=0.9, max_depth=7) #, objective ='reg:squarederror'
    model.fit(x_train, y_train)
    return model

In [60]:
#模型训练
model_xgb = build_model_xgb(x_train,y_train)
val_xgb = model_xgb.predict(x_val)
MAE_xgb = mean_absolute_error(y_val,val_xgb) #用平均绝对误差MAE来评价模型
print('MAE of val with xgb:',MAE_xgb)

Train xgb...
[22:13:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE of val with xgb: 953.8296908175865


* 平均绝对误差（Mean Absolute Error，MAE）越小，说明回归模型的拟合效果越好。MAE用于反映预测值与真实值误差的实际情况，其计算公式如下：
$$
MAE=\frac{1}{N} \sum_{i=1}^{N}\left|y_{i}-\hat{y}_{i}\right|
$$

* 在前面建立的模型中，MAE=953.8296908175865,值偏大。由于这里的样本数据有150000条，样本基数大，因此拟合效果还算可以。